In [19]:
# !pip install vit-pytorch
# !pip install performer-pytorch
# !pip install nystrom-attention
!pip install vit-pytorch


In [1]:
import torch
import torch.nn as nn
import math
from matplotlib import pyplot as plt
from tqdm import tqdm
from vit_pytorch.efficient import ViT
from performer_pytorch import Performer
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [13]:
device = torch.device(
        'cuda') if torch.cuda.is_available() else torch.device('cpu')
batch_size = 128
lr = 0.001
epochs = 3

transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

mnist_train = datasets.CIFAR10(root="CIFAR10/",
                             train=True,
                             download=True,
                             transform=transform)

mnist_test = datasets.CIFAR10(root="CIFAR10/",
                             train=False,
                             download=True,
                             transform=transform)

data_train = DataLoader(dataset=mnist_train,
                        batch_size=batch_size,
                        shuffle=True)

data_test= DataLoader(dataset=mnist_train,
                        batch_size=batch_size//4,
                        shuffle=False)



Files already downloaded and verified
Files already downloaded and verified


In [6]:
import torch
from vit_pytorch.efficient import ViT
from nystrom_attention import Nystromformer

In [22]:
# efficient_transformer =  Performer(
#     dim_head = 64,
#     dim = 512,
#     depth = 1,
#     heads = 8,
#     causal = True
# )

# model = ViT(
#     dim = 512,
#     image_size = 32,
#     patch_size = 16,
#     num_classes = 10,
#     transformer = efficient_transformer
# )

efficient_transformer = Nystromformer(
    dim = 512,
    depth = 12,
    heads = 8,
    num_landmarks = 128
)

v = ViT(
    dim = 512,
    image_size = 32,
    patch_size = 4,
    num_classes = 10,
    transformer = efficient_transformer
)

In [23]:
opt = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
model.to(device)
l = []
for epoch in range(1, epochs + 1):
    acc = 0
    tot_loss = 0
    train_cnt = 0
    test_cnt = 0
    model.train()
    pbar = tqdm(data_train)
    for x, y in pbar:
#         print(x.shape)
#         break
        x = x.to(device)
        y = y.to(device)
        y_pred = model(x)
        opt.zero_grad()
        loss = criterion(y_pred, y)
        loss.backward()
        opt.step()
        tot_loss += loss.item()*x.shape[0]
        train_cnt += x.shape[0]
        pbar.set_description(f"Loss : {tot_loss/train_cnt:.4f}")
#     break
    model.eval()

    for x, y in data_test:
        x = x.to(device)
        y = y.to(device)
        y_pred = model(x)

        y_argmax = y_pred.argmax(dim = 1)
        acc += (y == y_argmax).sum()
        test_cnt += x.shape[0]
    a = tot_loss/math.log(2)
    l.append(a)
    print(f'epoch {epoch} : Average loss : {tot_loss/train_cnt:.4f}, test_acc : {acc.item()/test_cnt:.4f}')

plt.plot(l)
plt.show()
print(l)

Loss : 2.3143: 100%|██████████| 391/391 [00:12<00:00, 30.14it/s]


KeyboardInterrupt: 

In [15]:
import torch
from performer_pytorch import Performer

model = Performer(
    dim = 512,
    dim_head = 8,
    depth = 1,
    heads = 8,
    causal = True
)

x = torch.randn(1, 2048, 512)
model(x) # (1, 2048, 512)

unable to import cuda code for auto-regressive Performer. will default to the memory inefficient non-cuda version


tensor([[[-0.7665, -1.2752,  0.5297,  ..., -1.6981, -0.7756,  0.1333],
         [-1.0223,  0.3121,  0.4439,  ..., -1.1971, -0.1929,  0.1666],
         [ 0.3963,  1.7769,  0.7451,  ..., -1.3259, -1.1830,  0.6916],
         ...,
         [-0.2059, -2.0350, -2.2680,  ..., -1.2640,  1.7713,  0.0140],
         [-0.6569,  1.2208, -1.1717,  ..., -0.1765, -0.0173,  0.2284],
         [-0.0964,  1.2774,  0.4486,  ..., -0.7397, -0.8127, -2.0359]]],
       grad_fn=<AddBackward0>)

In [8]:
!pip install performer-pytorch
